In [2]:
### import libraries

import wradlib as wrl
import rasterio
from rasterio.transform import Affine
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import datetime as dt
import logging
import sys
import requests

In [3]:
def download_data(date):

    #Which data you want to download (Volume radar for a whole day as tar file)

    # Set the API key. See https://developer.dataplatform.knmi.nl/get-started#obtain-an-api-key for more information
    api_key = "eyJvcmciOiI1ZTU1NGUxOTI3NGE5NjAwMDEyYTNlYjEiLCJpZCI6ImE1OGI5NGZmMDY5NDRhZDNhZjFkMDBmNDBmNTQyNjBkIiwiaCI6Im11cm11cjEyOCJ9"  # Public API key

    #date = "20230104"   # Specify the date in format yearmonthday all together
    dateObject = dt.datetime.strptime(date, '%Y%m%d')
    # Set up logging
    logging.basicConfig()
    logger = logging.getLogger(__name__)
    logger.setLevel("INFO")

    api_url = "https://api.dataplatform.knmi.nl/open-data"
    api_version = "v1"


    def main():
        # Parameters
        max_keys = "1"
        global filename
        #radar parameters
        dataset_name = "radar_forecast"
        filename = f"RAD62_OPER_O___TARVOL__L2__{date}T000000_{(dateObject + dt.timedelta(days=1)).strftime ('%Y%m%d')}T000000_0001.tar"
        dataset_version = "1.0"


        # Use list files request to request file of the day.
        get_file_response  = requests.get(
            f"https://api.dataplatform.knmi.nl/open-data/v1/datasets/radar_tar_vol_full_herwijnen/versions/1.0/files/{filename}/url",
            headers={"Authorization": api_key},
        )
        
        if get_file_response.status_code != 200:
            logger.error("Unable to retrieve download url for file")
            logger.error(get_file_response.text)
            sys.exit(1)

        logger.info(f"Successfully retrieved temporary download URL for dataset file {filename}")

        download_url = get_file_response.json().get("temporaryDownloadUrl")
        # Check logging for deprecation
        if "X-KNMI-Deprecation" in get_file_response.headers:
            deprecation_message = get_file_response.headers.get("X-KNMI-Deprecation")
            logger.warning(f"Deprecation message: {deprecation_message}")

        download_file_from_temporary_download_url(download_url, filename)


    def download_file_from_temporary_download_url(download_url, filename):
        try:
            with requests.get(download_url, stream=True) as r:
                r.raise_for_status()
                with open(filename, "wb") as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
        except Exception:
            logger.exception("Unable to download file using download URL")
            sys.exit(1)

        logger.info(f"Successfully downloaded dataset file to {filename}")

    
    if __name__ == "__main__":
        main()

In [4]:
for date in [20220206, 20220520, 20220928]:
    '''
    Loop over the dates and download the data for each date.
    '''    
    try: 
        # Download the data with KNMI API
        download_data(date)
    except:
        print('No data for', date)
        continue
    

INFO:__main__:Successfully retrieved temporary download URL for dataset file RAD62_OPER_O___TARVOL__L2__20220206T000000_20220207T000000_0001.tar
